In this notebook, I work Around Retrival Augmented Generation using the OpenAI API ad langchain. 
The idea is to convert an Excel file to an SQL database. Then query this database to look for answers to questions asked in Natural Language. 

So It is doing Natural Language to SQL conversion.

In [20]:
!pip install -q langchain
!pip install -q openai

In [40]:
import os
os.environ['OPENAI_API_KEY'] = 'Key'

In [51]:

import numpy as np #
import pandas as pd #
import sqlite3 #
df = pd.read_excel("/content/2024-01-18T03_19_00+0000.xlsx")


In [52]:
df.columns

Index(['_File', 'CERTIFICATE', 'CHARACTERISTIC', 'COMPANY', 'DATE',
       'EDUCATION', 'EMAIL', 'INDUSTRY', 'JOBTITLE', 'LANGUAGES', 'LOCATION',
       'NAME', 'NUMBER', 'SKILL', 'TIME', 'URL', 'Text'],
      dtype='object')

In [ ]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('People.sqlite')
c = conn.cursor()

# Create the People table if it doesn't exist
c.execute('''CREATE TABLE IF NOT EXISTS People (
    _File TEXT,
    CERTIFICATE TEXT,
    CHARACTERISTIC TEXT,
    COMPANY TEXT,
    DATE TEXT,
    EDUCATION TEXT,
    EMAIL TEXT,
    INDUSTRY TEXT,
    JOBTITLE TEXT,
    LANGUAGES TEXT,
    LOCATION TEXT,
    NAME TEXT,
    NUMBER TEXT,
    SKILL TEXT,
    TIME TEXT,
    URL TEXT,
    Text TEXT
)''')
conn.commit()

# Insert data from the DataFrame into the People table
df.to_sql('People', conn, if_exists='replace', index=False)

# Retrieve and print all rows from the People table
c.execute('''SELECT * FROM People''')
for row in c.fetchall():
    print(row)

# Close the cursor and database connection
#c.close()
#conn.close()


In [ ]:
import sqlite3

def read_sql_query(sql, db):
    conn = sqlite3.connect(db)
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    conn.close()
    return rows

# Example usage
db_file = 'People.sqlite'
sql_query = 'SELECT * FROM People'
result_rows = read_sql_query(sql_query, db_file)

for row in result_rows:
    print(row)


In [ ]:
rows = read_sql_query('SELECT * FROM People LIMIT 10;', "People.sqlite")
for row in rows:
    print(row)

In [ ]:
!pip install --upgrade langchain


In [62]:
from langchain.llms import OpenAI
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [63]:
# Create an instance of SQLDatabase from the SQLite database file
input_db = SQLDatabase.from_uri('sqlite:///People.sqlite')

# Create an instance of OpenAI with a temperature of 0
llm_1 = OpenAI(temperature=0)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [64]:
db_agent = SQLDatabaseChain(llm = llm_1, database = input_db, verbose=True)

/usr/local/lib/python3.10/dist-packages/langchain_experimental/sql/base.py:76: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [65]:
db_agent.run("how many rows are there?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
how many rows are there?
SQLQuery:SELECT COUNT(*) FROM People
SQLResult: [(21130,)]
Answer:There are 21130 rows in the People table.
> Finished chain.


'There are 21130 rows in the People table.'

In [69]:
db_agent.run("Give me the degree of the first row")



> Entering new SQLDatabaseChain chain...
Give me the degree of the first row
SQLQuery:SELECT "EDUCATION" FROM "People" LIMIT 1
SQLResult: [('Computer Science graduate',)]
Answer:

'Computer Science graduate'